labels.npy corrisponde alla struttura che contiene le info di classe, oggetto e posizione sulla griglia. Nel caso del codice che ho su github, 

 0 = row_id, 1 = col_id, 2 = class_id, colonna 3 = object_id

In [1]:
import numpy as np
data = np.load('SITS-Missing-Data/D1_balaruc_samples.npy')
masks = np.load('SITS-Missing-Data/D2_balaruc_masks.npy')
lut = np.load('SITS-Missing-Data/D3_balaruc_lut.npy')

In [2]:
data.shape, masks.shape, lut.shape

((137606, 54, 16), (137606, 54), (137606, 4))

In [23]:
def get_unique_ids_by_class(lut):
    labels = lut[:, 1]
    unique_labels = np.unique(labels)
    ids_by_class = {}
    for label in unique_labels:
      idx = np.where(labels == label)
      lut_subset = lut[idx]
      ids_by_class[label] = np.unique(lut_subset[:, 0])
    return ids_by_class

In [22]:
from sklearn.utils import shuffle

In [54]:
def get_idx_of_object_ids(ids, lut):
    lut_ids = lut[:, 0]
    tot_idx = []
    for i in ids:
        tot_idx.append(np.where(lut_ids == i)[0])
    tot_idx = np.concatenate(tot_idx, axis=0)
    return tot_idx

In [68]:
np.where(lut[:, 0] == 5)

(array([73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84]),)

In [70]:
def get_split_idx(lut, train_perc=.6, val_perc=.2):
  train_idx, valid_idx, test_idx = [], [], []
  unique_ids_by_class = get_unique_ids_by_class(lut)

  for label in unique_ids_by_class:
    ids = unique_ids_by_class[label]
    ids = shuffle(ids)
    
    limit_train = int(len(ids)* train_perc )
    limit_val = limit_train + int(len(ids)* val_perc)
    
    train_idx.extend(get_idx_of_object_ids(ids[0:limit_train], lut))
    valid_idx.extend(get_idx_of_object_ids(ids[limit_train:limit_val], lut))
    test_idx.extend(get_idx_of_object_ids(ids[limit_val::], lut))
  return (train_idx,), (valid_idx,), (test_idx,)
    


In [71]:
train_idx, valid_idx, test_idx = get_split_idx(lut)


In [72]:
train_idx

([8819,
  8820,
  8821,
  8822,
  8823,
  8824,
  9918,
  9919,
  9920,
  9921,
  9922,
  9923,
  9924,
  9925,
  9926,
  9927,
  9928,
  9929,
  9930,
  9931,
  9932,
  9933,
  9934,
  9935,
  6668,
  6669,
  6670,
  6671,
  6672,
  6673,
  6674,
  6675,
  6676,
  6677,
  6678,
  6679,
  6680,
  6681,
  6682,
  6683,
  6684,
  6685,
  6686,
  6687,
  6688,
  6689,
  6690,
  6691,
  6692,
  6693,
  6694,
  6695,
  6696,
  6697,
  6698,
  6699,
  6700,
  6701,
  6702,
  6703,
  6704,
  6705,
  6706,
  6707,
  6708,
  6709,
  6710,
  9234,
  9235,
  9236,
  9237,
  9238,
  9239,
  9240,
  9241,
  9242,
  9243,
  9244,
  9245,
  9246,
  9247,
  9248,
  9249,
  1757,
  1758,
  1759,
  1760,
  1761,
  1762,
  1763,
  1764,
  1765,
  1766,
  1767,
  1768,
  1769,
  1770,
  1771,
  1772,
  1773,
  1774,
  1775,
  1776,
  1777,
  1778,
  1779,
  1780,
  1781,
  1782,
  2044,
  2045,
  2046,
  2047,
  2048,
  2049,
  2050,
  2051,
  2052,
  2053,
  2054,
  7466,
  7467,
  7468,
  7469,
  7470,


In [61]:
from utils import *

In [75]:
def generate_dataset(idx, data, prediction_target, mask, labels, num_classes):
    data_subset = data[idx]
    mask_subset = mask[idx]
    labels_subset = convert_to_one_hot(labels[idx], num_classes)
    prediction_target_subset = prediction_target[idx]
    return tf.data.Dataset.from_tensor_slices((data_subset, mask_subset, labels_subset, prediction_target_subset))

In [62]:
data[np.where(masks == 1)] = MISSING_VALUE

num_steps = data.shape[1]
num_bands = data.shape[2]
labels, num_classes = transfer_labels(lut[:, 1])
prediction_target = data[:, 1:]
mask = np.ones_like(prediction_target)
mask[np.where(prediction_target == MISSING_VALUE)] = 0

check_missing_ratio(np.array(mask))

data = data.reshape(-1, num_steps, num_bands)
prediction_target = prediction_target.reshape(-1, num_steps - 1, num_bands)
mask = mask.reshape(-1, num_steps - 1, num_bands)

MISSING RATIO: 0.339


In [76]:
train_dataset = generate_dataset(train_idx, data, prediction_target, mask, labels, num_classes)

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-09-01 16:33:28.302408: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-01 16:33:28.303445: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [77]:
train_dataset

<TensorSliceDataset element_spec=(TensorSpec(shape=(54, 16), dtype=tf.float32, name=None), TensorSpec(shape=(53, 16), dtype=tf.float32, name=None), TensorSpec(shape=(8,), dtype=tf.int32, name=None), TensorSpec(shape=(53, 16), dtype=tf.float32, name=None))>